In [2]:
import numpy as np
import yaml
from astropy.table import Table
from astropy.io import fits
import desispec.maskbits as masks
import desimodel.io

# YAML target description

In [3]:
stream = open('../../desidata/inputs/targets.yaml', 'r')
_alldefs = yaml.load(stream)
stream.close()
attr = []
for line in _alldefs["targetmask"]:
    attr.append(line[:3])
_targetbitdefs = dict([("targetmask", attr)])
print _targetbitdefs

{'targetmask': [['QSOLYA', 0, 'Lyman alpha QSO'], ['QSO', 1, 'Tracer QSO'], ['LRG', 2, 'Luminous Red Galaxy'], ['ELG', 3, 'Emission Line Galaxy'], ['STAR', 4, 'Standard Star'], ['SKY', 5, 'Sky'], ['BGS', 6, 'Bright Galaxy']]}


In [4]:
#Mock set of random objects

In [5]:
targetmask = masks.BitMask("targetmask", _targetbitdefs)
n_tiles = 10
target_per_tile = 25000
n_fibers = 5000
total_n_targets = target_per_tile * n_tiles / 2
random_ID_list =  np.arange(total_n_targets, dtype='int64')
random_ra_list = np.random.random(total_n_targets)
random_dec_list = np.random.random(total_n_targets)
random_type_A_list = np.int_(np.random.random(total_n_targets)*7)
random_type_B_list = np.int_(np.random.random(total_n_targets)*7)
print np.size(np.where(random_type_A_list == random_type_B_list))

17970


# DESI targets

In [6]:
def write_target_fits(id_list, ra_list, dec_list, type_list, filename):
    t = Table([id_list, ra_list, dec_list, type_list], names=('ID', 'RA', 'DEC', 'TYPE'), meta={'name': 'target table'})
    t.write(filename, format='fits', overwrite=True)
    
for i_tile in range(n_tiles):
    #choose randomly targets on different tiles
    filename = 'tmp/targets_%06d.fits'%(i_tile)
    tile_id = np.random.choice(random_ID_list, replace=False,size=target_per_tile)
    tile_ra = random_ra_list[tile_id]
    tile_dec = random_dec_list[tile_id]
    tile_type_A = random_type_A_list[tile_id]
    tile_type_B = random_type_B_list[tile_id] 
    tile_type = np.ones(target_per_tile, dtype='int64')
    
    #create mock types
    for i in range(target_per_tile):
        tile_type[i]  = targetmask.mask(random_type_A_list[i]) | targetmask.mask(random_type_B_list[i])
        
    #write and read
    write_target_fits(tile_id, tile_ra, tile_dec, tile_type, filename)
    t = Table.read(filename, format='fits')

# DESI observations

Tracks what objects have been targetted.
* Object-id (int)
* ObservedFlag (int) Yes/No (1/0) flag.

In [7]:
def write_observations_fits(id_list, observed_list, filename):
    t = Table([id_list, observed_list], names=('ID', 'OBSFLAG'), meta={'name': 'observations table'})
    t.write(filename, format='fits', overwrite=True)

for i_tile in range(n_tiles):
    filename = 'tmp/targets_%06d.fits'%(i_tile)
    t = Table.read(filename, format='fits')
    
    object_id = t['ID']
    observed_id = np.random.choice(np.arange(np.size(object_id)), replace=False,size=n_fibers)
    observed_flag = np.zeros(np.size(object_id), dtype=int)
    observed_flag[observed_id] = 1
    print sum(observed_flag), np.size(observed_id), observed_id
    
    filename = 'tmp/observations_%06d.fits'%(i_tile)
    write_observations_fits(object_id, observed_flag, filename)
    

5000 5000 [14851  3444 23401 ..., 22305  9812 10724]
5000 5000 [18340  1535  3501 ...,  2515 23239 12032]
5000 5000 [ 5031  6897 21455 ..., 17747  3018 13122]
5000 5000 [11981  2138 12089 ...,  4367  5662 23873]
5000 5000 [24859  8101 22376 ..., 19771 20557  1224]
5000 5000 [12116  5049 20715 ...,  4384 15748 23461]
5000 5000 [13676  6928 22621 ...,  7168 24148 22473]
5000 5000 [23390 14360  3674 ...,  9973  4677 14197]
5000 5000 [20282 23424 16442 ..., 12416 13815 20640]
5000 5000 [15361 10300 14803 ...,  7381  8512  5738]


# DESI results

Tracks the results of processed observed tiles

* Object-id (int)
* DesiType (int64) Mask
* z_spec (float)

In [8]:
def write_results_fits(id_list, type_list, redshift_list, filename):
    t = Table([id_list, type_list, redshift_list], names=('ID', 'TYPE', 'REDSHIFT'), meta={'name': 'results table'})
    t.write(filename, format='fits', overwrite=True)

#generates mock results for each tile
for i_tile in range(n_tiles):
    target_file = 'tmp/targets_%06d.fits'%(i_tile)
    observations_file = 'tmp/observations_%06d.fits'%(i_tile)
    targets = Table.read(target_file, format='fits')
    observations = Table.read(observations_file, format='fits')
    
    
    targets_id = targets['ID']
    observations_id = observations['ID']
    n_targets = np.size(targets_id)
    
    assert np.size(targets_id)==np.size(observations_id)
    
    z_list = np.zeros(n_targets)
    type_list  = targets['TYPE'].copy()
    for i in range(n_targets):
        z_list[i] = -1.0
        type_list[i] = targets['TYPE'][i]
        if(observations['OBSFLAG'][i]==1):
            z_list[i] = np.random.random()
            type_bit_offset = np.int_(np.random.random()*7)
            type_list[i] = targetmask.mask(type_bit_offset)
    
    filename = 'tmp/results_%06d.fits'%(i_tile)
    write_results_fits(targets_id, type_list, z_list, filename)

# Merged Target List

Uses all the targets/observations/results files to produce a single file to be fed into fiber assignment.* *

* Object-id (int)
* RA (float)
* DEC (float)
* FiberPriority (int)
* FiberObservationsNeeded (int)

In [9]:
targetmask.names()
print targetmask.mask('QSO')
print 'bitnum', targetmask.bitnum('QSO')
print targetmask.mask('LRG')
print _alldefs['targetmask'][targetmask.bitnum('STAR')][4], 'priority'
print _alldefs['targetmask'][targetmask.bitnum('STAR')][5], 'nobs'

2
bitnum 1
4
4 priority
1 nobs


In [10]:
def write_results_fits(id_list, ra_list, dec_list, priority_list, nobs_list, filename):
    t = Table([id_list, ra_list, dec_list, priority_list, nobs_list],
              names=('ID', 'RA', 'DEC', 'PRIOR', 'NOBS'), meta={'name': 'MTL table'})
    t.write(filename, format='fits', overwrite=True)
    
for i_tile in range(n_tiles):
    targets_filename = 'tmp/targets_%06d.fits'%(i_tile)
    observations_filename = 'tmp/observations_%06d.fits'%(i_tile)
    results_filename = 'tmp/results_%06d.fits'%(i_tile)
    mtl_filename = 'tmp/mtl_%06d.fits'%(i_tile)
    
    targets = Table.read(targets_filename, format='fits')
    observations = Table.read(observations_filename, format='fits')
    results = Table.read(results_filename, format='fits')

    
    n_points  = np.size(observations['ID'])
    
    mtl_type = np.ones(n_points, dtype='int64')
    mtl_priority = np.ones(n_points, dtype='int')
    mtl_nobs = np.ones(n_points, dtype='int')
    mtl_id = np.ones(n_points, dtype='int64')
    mtl_ra = np.ones(n_points)
    mtl_dec = np.ones(n_points)
    mtl_ra = targets['RA']
    mtl_dec = targets['DEC']
    mtl_id = targets['ID']
    for i in range(n_points):
        for name in targetmask.names():
            if((targets['TYPE'][i] & targetmask.mask(name)) != 0):
                break
        mtl_type[i] = targetmask.mask(name)
        mtl_priority[i] = _alldefs['targetmask'][targetmask.bitnum(name)][4]
        mtl_nobs[i] = _alldefs['targetmask'][targetmask.bitnum(name)][4] - observations['OBSFLAG'][i]
        
    write_results_fits(mtl_id, mtl_ra, mtl_dec, mtl_priority, mtl_nobs, mtl_filename)

# Fiber assignment

In [11]:
def write_assign_fits(id_list, ra_list, dec_list, fiber_list, filename):
    t = Table([id_list, ra_list, dec_list, fiber_list],
              names=('ID', 'RA', 'DEC', 'FIBERID'), meta={'name': 'Assign table'})
    t.write(filename, format='fits', overwrite=True)

#load fibers
fibers = desimodel.io.load_fiberpos()
fiberid = fibers['FIBER']
n_fibers = np.size(fiberid)

for i_tile in range(n_tiles):
    mtl_filename = 'tmp/mtl_%06d.fits'%(i_tile)
    assign_filename = 'tmp/assignment_%06d.fits'%(i_tile)

    mtl = Table.read(mtl_filename, format='fits')
    n_points = np.size(mtl['ID'])
    selection = np.random.choice(np.arange(n_points), replace=False,size=n_fibers)
    assign_id  = mtl['ID'][selection]
    assign_ra  = mtl['RA'][selection]
    assign_dec = mtl['DEC'][selection]
    assign_fiber = fiberid

    write_assign_fits(assign_id, assign_ra, assign_dec, assign_fiber, assign_filename)

In [12]:
def write_available_fits(id_list, fiber_list, filename):
    t = Table([id_list, fiber_list],
              names=('ID', 'FIBERID'), meta={'name': 'Available table'})
    t.write(filename, format='fits', overwrite=True)
    
#load fibers
fibers = desimodel.io.load_fiberpos()
fiberid = fibers['FIBER']
n_fibers = np.size(fiberid)

for i_tile in range(n_tiles):
    mtl_filename = 'tmp/mtl_%06d.fits'%(i_tile)
    assign_filename = 'tmp/assignment_%06d.fits'%(i_tile)
    available_filename = 'tmp/available_%06d.fits'%(i_tile)
    mtl = Table.read(mtl_filename, format='fits')

    available_id = np.empty((0))
    available_fiber = np.empty((0))
    for i in range(n_fibers):
        n_points = np.random.choice(np.arange(6))
        
        if(n_points!=0):
            selection = np.random.choice(np.arange(n_points), replace=False,size=n_points)
            available_id = np.append(available_id, selection)
            available_fiber = np.append(available_fiber, np.ones(n_points, dtype='int')*fiberid[i])

    write_available_fits(available_id, available_fiber, available_filename)